In [1]:
""" MNIST 
    MLP 
    MODEL"""

# import
import os
import tensorflow as tf
import numpy as np

In [2]:
# Set Placeholders

X = tf.placeholder(tf.float32, shape=[None, 28*28], name='X')
Y = tf.placeholder(tf.int64, shape=[None], name='Y')
Y_onehot = tf.one_hot(Y, 10, axis=1, name='Y_onehot')

training1 = tf.placeholder(tf.bool, name = 'training1')

# Set Variables
# Set hidden Layer1
with tf.name_scope("Layer1"):
    W1 = tf.Variable(tf.truncated_normal([784, 300], stddev=0.02), name='W1')
    B1 = tf.Variable(tf.zeros([300]), name = 'B1')
    H1 = tf.matmul(X, W1) + B1
    H1 = tf.nn.relu(H1, name='relu1')
    H1 = tf.layers.dropout(H1, rate=0.5, training = training1, name='train1')

# Set hidden Layer2

with tf.name_scope("Layer2"):
    W2 = tf.Variable(tf.truncated_normal([300,200], stddev=0.02), name='W2')
    B2 = tf.Variable(tf.zeros([200]), name='B2')
    H2 = tf.matmul(H1, W2) + B2
    H2 = tf.nn.relu(H2, name='relu2')
    H2 = tf.layers.dropout(H2, rate=0.5, training = training1, name='train2' )

# Set Last Layer

with tf.name_scope("Layer3"):
    W3 = tf.Variable(tf.truncated_normal([200,10], stddev=0.02), name='W3')
    B3 = tf.Variable(tf.zeros([10]), name='B3')

    Y_pred = tf.matmul(H2, W3) + B3
    Y_pred = tf.layers.dropout(Y_pred, rate=0.5, training = training1, name='lastLayer' )

    # Set Loss Function(Softmax)

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Y_pred, labels=Y_onehot))

# Set SoftMax & Accuracy Function

with tf.name_scope("Softmax"):
    softmax = tf.nn.softmax(logits=Y_pred)
    correct = tf.equal(tf.argmax(softmax,1), Y)
    acc = tf.reduce_mean(tf.cast(correct, tf.float32))

# Set Trainer & Optimizer

with tf.name_scope('Optimizer'):
    trainer = tf.train.GradientDescentOptimizer(learning_rate=0.05, name='trainer')
    optimizer = trainer.minimize(loss, name='optimizer')
    

tf.summary.scalar('loss', loss)
tf.summary.scalar('accuracy',acc)
summary_op = tf.summary.merge_all()

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

!rm -fvr logdir


removed ‘logdir/events.out.tfevents.1504842330.student13’
removed directory: ‘logdir’


In [11]:
# Data Loading & Normalization
# image file has pixel data from 0016 -> reshape(60000,28,28), float
# label file has label data from 0008 -> reshape(60000), int

fd = open('./mnist/train-images.idx3-ubyte')
images = np.fromfile(file=fd, dtype=np.uint8)
images = images[16:].reshape([60000,28*28]).astype(np.float)
images = images/127. -1.

#print(images[1,])
fd = open('./mnist/train-labels.idx1-ubyte')
labels = np.fromfile(file=fd, dtype=np.uint8)
labels = labels[8:].reshape([60000]).astype(np.int)

# Test Data Loading
fd = open('./mnist/t10k-images.idx3-ubyte')
t_images = np.fromfile(file=fd, dtype=np.uint8)
t_images = t_images[16:].reshape([10000,28*28]).astype(np.float)
t_images = t_images/127. -1.

fd = open('./mnist/t10k-labels.idx1-ubyte')
t_labels = np.fromfile(file=fd, dtype=np.uint8)
t_labels = t_labels[8:].reshape([10000]).astype(np.int)

# Session Create

sess = tf.InteractiveSession(config=config)
    
# variable initialize
sess.run(tf.global_variables_initializer())
    
# Set batch size
batch_size = 500
batch_count = 60000 // batch_size
test_count = 10000 // batch_size
    
writer = tf.summary.FileWriter('logdir', graph = tf.get_default_graph())
    
step = 1
# Epoch Loop
for epoch in range(50) :
        
    # Set loss, acc, t_acc
    total_loss = 0.
    total_acc = 0.
    total_t_acc = 0.
    # Mini-Batch Loop
    for i in range(batch_count):
           
        # Set batch_index
        batch_index = i * batch_size
        
        img = np.reshape(images[batch_index : batch_index + batch_size], [batch_size, 784])
        label = labels[batch_index : batch_index + batch_size]
            
        # Session run
        _, loss_v, acc_v, summary_v = sess.run([optimizer, loss, acc, summary_op], feed_dict={X:img, Y:label, training1:True})
            
        total_loss += loss_v
        total_acc += acc_v
            
        writer.add_summary(global_step=step, summary=summary_v)
        step += 1
        
    total_loss = total_loss / batch_count
    total_acc= total_acc / batch_count
        
    # Set Test Loop
    for j in range(test_count) :
            
        # Set batch_index
        test_index = j * batch_size
            
        t_img = np.reshape(t_images[test_index : test_index + batch_size], [batch_size, 784])
        t_label = t_labels[test_index : test_index + batch_size]
            
        # Session run
        t_acc_v, summary_t = sess.run([acc, summary_op], feed_dict={X:t_img, Y:t_label, training1:False})
            
        total_t_acc += t_acc_v
            
        writer.add_summary(global_step=step, summary=summary_t)
        step += 1
        
    total_t_acc = total_t_acc / test_count
        
    print('tot_Loss = %.5f, Train_Accuracy = %.5f, Test Accuracy = %.2f%%' % (total_loss, total_acc, total_t_acc*100) )
        
sess.close()

tot_Loss = 2.19926, Train_Accuracy = 0.21980, Test Accuracy = 61.77%
tot_Loss = 1.67986, Train_Accuracy = 0.40578, Test Accuracy = 81.72%
tot_Loss = 1.43103, Train_Accuracy = 0.47525, Test Accuracy = 86.05%
tot_Loss = 1.32421, Train_Accuracy = 0.50607, Test Accuracy = 88.47%
tot_Loss = 1.26276, Train_Accuracy = 0.51938, Test Accuracy = 89.69%
tot_Loss = 1.22163, Train_Accuracy = 0.52972, Test Accuracy = 90.70%
tot_Loss = 1.18694, Train_Accuracy = 0.53602, Test Accuracy = 91.11%
tot_Loss = 1.16005, Train_Accuracy = 0.54240, Test Accuracy = 91.57%
tot_Loss = 1.15073, Train_Accuracy = 0.54388, Test Accuracy = 92.37%
tot_Loss = 1.11946, Train_Accuracy = 0.55058, Test Accuracy = 92.66%
tot_Loss = 1.11206, Train_Accuracy = 0.55175, Test Accuracy = 93.02%
tot_Loss = 1.08910, Train_Accuracy = 0.55863, Test Accuracy = 93.30%
tot_Loss = 1.07995, Train_Accuracy = 0.56000, Test Accuracy = 93.69%
tot_Loss = 1.07887, Train_Accuracy = 0.55805, Test Accuracy = 93.71%
tot_Loss = 1.06537, Train_Accuracy

In [7]:
!tensorboard --logdir logdir

W0908 12:50:43.432061 Reloader plugin_event_accumulator.py:303] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
TensorBoard 0.1.5 at http://student13:6006 (Press CTRL+C to quit) ^C

